In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform

In [2]:
club = pd.read_csv('recommend.csv')
club.tail()

,ID,Name,Position,Age,Height(cm),Weight(kgs),Preferred Foot,Weak Foot,Nationality,Continent,...,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB
18202,238813,J. Lundstram,CM,19,179.823225,60.794728,Right,2.0,England,Europe,...,44,45,45,45,44,45,45,45,45,45
18203,243165,N. Christoffersson,ST,19,192.014630,77.127640,Right,2.0,Sweden,Europe,...,30,31,31,31,30,29,32,32,32,29
18204,241638,B. Worman,ST,16,176.775373,67.146416,Right,3.0,England,Europe,...,34,30,30,30,34,33,28,28,28,33
18205,246268,D. Walker-Rice,RW,17,155.440415,69.868568,Right,3.0,England,Europe,...,36,32,32,32,36,35,31,31,31,35
18206,246269,G. Nugent,CM,16,155.440415,79.849792,Right,3.0,England,Europe,...,46,46,46,46,46,46,47,47,47,46


In [3]:
recom_club = club.copy()

In [4]:
recom_club = club.drop(columns=['ID','Club','Position','Preferred Foot', 'Nationality','Continent',
                        'Joined', 'Evolution','Contract_Expiry','Work Rate','Work RAttack',  
                     'Work RDefense','year_joined', 'month_joined','day_joined','Age', 'Height(cm)', 
                                'Weight(kgs)', 'Weak Foot', 'Inter_Rep', 'Jersey', 'Value(€M)', 'Wage(€K)',
       'Release_Clause(€M)'])
recom_club = recom_club.set_index('Name').T

In [5]:
recom_club

Name,L. Messi,Cristiano Ronaldo,Neymar Jr,De Gea,K. De Bruyne,E. Hazard,L. Modrić,L. Suárez,Sergio Ramos,J. Oblak,...,D. Holland,J. Livesey,M. Baldisimo,J. Young,D. Walsh,J. Lundstram,N. Christoffersson,B. Worman,D. Walker-Rice,G. Nugent
Overall,94.0,94.0,92.0,91.0,91.0,91.0,91.0,91.0,91.0,90.0,...,47.0,47.0,47.0,47.0,47.0,47.0,47.0,47.0,47.0,46.0
Potential,94.0,94.0,93.0,93.0,92.0,91.0,91.0,91.0,91.0,93.0,...,61.0,70.0,69.0,62.0,68.0,65.0,63.0,67.0,66.0,66.0
Difference,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,3.0,...,14.0,23.0,22.0,15.0,21.0,18.0,16.0,20.0,19.0,20.0
Special,2202.0,2228.0,2143.0,1471.0,2281.0,2142.0,2280.0,2346.0,2201.0,1331.0,...,1362.0,792.0,1303.0,1203.0,1098.0,1307.0,1098.0,1189.0,1228.0,1321.0
SkillMoves,4.0,5.0,5.0,1.0,4.0,4.0,4.0,3.0,3.0,1.0,...,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LB,59.0,61.0,60.0,50.0,73.0,60.0,79.0,66.0,84.0,50.0,...,44.0,50.0,47.0,30.0,42.0,45.0,29.0,33.0,35.0,46.0
LCB,47.0,53.0,47.0,50.0,66.0,49.0,71.0,63.0,87.0,50.0,...,40.0,50.0,48.0,27.0,46.0,45.0,32.0,28.0,31.0,47.0
CB,47.0,53.0,47.0,50.0,66.0,49.0,71.0,63.0,87.0,50.0,...,40.0,50.0,48.0,27.0,46.0,45.0,32.0,28.0,31.0,47.0
RCB,47.0,53.0,47.0,50.0,66.0,49.0,71.0,63.0,87.0,50.0,...,40.0,50.0,48.0,27.0,46.0,45.0,32.0,28.0,31.0,47.0


In [6]:
recom_club.index

Index(['Overall', 'Potential', 'Difference', 'Special', 'SkillMoves',
       'ShortPass', 'LongPass', 'ShotPower', 'LongShots', 'Penalties',
       'Agility', 'Acceleration', 'SprintSpeed', 'Stamina', 'Strength',
       'Reactions', 'Dribbling', 'BallControl', 'Aggression', 'Interceptions',
       'Positioning', 'Vision', 'Balance', 'Composure', 'Crossing',
       'Finishing', 'Jumping', 'HeadAccuracy', 'Marking', 'StandingTackle',
       'SlidingTackle', 'GKDiving', 'Volleys', 'Curve', 'FKAccuracy',
       'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes', 'LS', 'ST',
       'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM', 'LM', 'LCM',
       'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB',
       'RCB', 'RB'],
      dtype='object')

In [7]:
distancias=squareform(pdist(recom_club.T, 'euclidean'))
distancias

array([[   0.        ,   69.38299503,   71.56116265, ..., 1058.25233286,
        1018.61965424,  928.9822388 ],
       [  69.38299503,    0.        ,  107.9212676 , ..., 1081.32788737,
        1042.44712096,  951.17611408],
       [  71.56116265,  107.9212676 ,    0.        , ...,  995.84285909,
         956.421978  ,  866.35847084],
       ...,
       [1058.25233286, 1081.32788737,  995.84285909, ...,    0.        ,
          73.30757123,  164.43235691],
       [1018.61965424, 1042.44712096,  956.421978  , ...,   73.30757123,
           0.        ,  126.87789406],
       [ 928.9822388 ,  951.17611408,  866.35847084, ...,  164.43235691,
         126.87789406,    0.        ]])

In [8]:
similitud=pd.DataFrame(1/(1+distancias), index=recom_club.columns, columns=recom_club.columns)
similitud.head()

Name,L. Messi,Cristiano Ronaldo,Neymar Jr,De Gea,K. De Bruyne,E. Hazard,L. Modrić,L. Suárez,Sergio Ramos,J. Oblak,...,D. Holland,J. Livesey,M. Baldisimo,J. Young,D. Walsh,J. Lundstram,N. Christoffersson,B. Worman,D. Walker-Rice,G. Nugent
Name,,,,,,,,,,,,,,,,,,,,,
L. Messi,1.000000,0.014208,0.013781,0.001230,0.007639,0.012920,0.006529,0.005713,0.004828,0.001053,...,0.001132,0.000683,0.001053,0.000955,0.000857,0.001059,0.000867,0.000944,0.000981,0.001075
Cristiano Ronaldo,0.014208,1.000000,0.009181,0.001200,0.008630,0.009006,0.006893,0.006898,0.005348,0.001031,...,0.001102,0.000672,0.001029,0.000934,0.000842,0.001034,0.000850,0.000924,0.000958,0.001050
Neymar Jr,0.013781,0.009181,1.000000,0.001330,0.005875,0.029530,0.005472,0.004475,0.004964,0.001126,...,0.001218,0.000713,0.001128,0.001015,0.000906,0.001134,0.000916,0.001003,0.001044,0.001153
De Gea,0.001230,0.001200,0.001330,1.000000,0.001132,0.001331,0.001133,0.001071,0.001246,0.006468,...,0.004072,0.001430,0.003661,0.002849,0.002283,0.003685,0.002274,0.002746,0.002979,0.003852
K. De Bruyne,0.007639,0.008630,0.005875,0.001132,1.000000,0.006035,0.016962,0.009810,0.006444,0.000979,...,0.001044,0.000650,0.000980,0.000887,0.000807,0.000983,0.000812,0.000878,0.000911,0.000998


In [9]:
tom = {'Overall': 75, 'Potential': 94,'Difference': 25, 'Special': 2202, 'SkillMoves': 3, 'ShortPass': 88, 'LongPass': 56,
       'ShotPower': 45, 'LongShots': 67, 'Penalties': 75, 'Agility': 65, 'Acceleration': 39,
       'SprintSpeed': 67,'Stamina': 58, 'Strength': 73, 'Reactions': 39, 'Dribbling': 64,
       'BallControl': 84, 'Aggression': 15, 'Interceptions': 47, 'Positioning': 89, 'Vision': 88,
       'Balance': 65, 'Composure': 72, 'Crossing': 64, 'Finishing': 69, 'Jumping': 41,
       'HeadAccuracy': 67, 'Marking': 70, 'StandingTackle': 26, 'SlidingTackle': 34, 'GKDiving': 31,
       'Volleys': 45, 'Curve': 31, 'FKAccuracy': 75, 'GKHandling': 34, 'GKKicking': 29,'GKPositioning': 10,
       'GKReflexes': 78, 'LS':65, 'ST':57, 'RS':87,'LW':58, 'LF':68,'CF':78, 'RF':88,'RW':58, 'LAM':78, 
       'CAM':89, 'RAM': 70 ,'LM':72, 'LCM':68, 'CM':77, 'RCM':57, 'RM':69, 'LWB':57, 'LDM':86, 'CDM':56, 
      'RDM':66, 'RWB':87, 'LB':56,'LCB':65, 'CB':75, 'RCB':65, 'RB':76}

In [11]:
recom_club['tom'] = pd.Series(tom)
#recom_club = recom_club.append(pd.Series(tom),ignore_index=True)

In [19]:
distance = squareform(pdist(recom_club.T, 'euclidean'))
similarity = pd.DataFrame(1/(1+distance), index=recom_club.columns, columns=recom_club.columns)
similarity

Name,L. Messi,Cristiano Ronaldo,Neymar Jr,De Gea,K. De Bruyne,E. Hazard,L. Modrić,L. Suárez,Sergio Ramos,J. Oblak,...,J. Livesey,M. Baldisimo,J. Young,D. Walsh,J. Lundstram,N. Christoffersson,B. Worman,D. Walker-Rice,G. Nugent,tom
Name,,,,,,,,,,,,,,,,,,,,,
L. Messi,1.000000,0.014208,0.013781,0.001230,0.007639,0.012920,0.006529,0.005713,0.004828,0.001053,...,0.000683,0.001053,0.000955,0.000857,0.001059,0.000867,0.000944,0.000981,0.001075,0.004870
Cristiano Ronaldo,0.014208,1.000000,0.009181,0.001200,0.008630,0.009006,0.006893,0.006898,0.005348,0.001031,...,0.000672,0.001029,0.000934,0.000842,0.001034,0.000850,0.000924,0.000958,0.001050,0.004858
Neymar Jr,0.013781,0.009181,1.000000,0.001330,0.005875,0.029530,0.005472,0.004475,0.004964,0.001126,...,0.000713,0.001128,0.001015,0.000906,0.001134,0.000916,0.001003,0.001044,0.001153,0.004929
De Gea,0.001230,0.001200,0.001330,1.000000,0.001132,0.001331,0.001133,0.001071,0.001246,0.006468,...,0.001430,0.003661,0.002849,0.002283,0.003685,0.002274,0.002746,0.002979,0.003852,0.001288
K. De Bruyne,0.007639,0.008630,0.005875,0.001132,1.000000,0.006035,0.016962,0.009810,0.006444,0.000979,...,0.000650,0.000980,0.000887,0.000807,0.000983,0.000812,0.000878,0.000911,0.000998,0.004900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
N. Christoffersson,0.000867,0.000850,0.000916,0.002274,0.000812,0.000917,0.000812,0.000776,0.000870,0.003023,...,0.002857,0.004298,0.007594,0.009385,0.004319,1.000000,0.008608,0.006681,0.004095,0.000882
B. Worman,0.000944,0.000924,0.001003,0.002746,0.000878,0.001004,0.000878,0.000836,0.000945,0.003620,...,0.002286,0.006519,0.017275,0.006642,0.006622,0.008608,1.000000,0.013458,0.006045,0.000959
D. Walker-Rice,0.000981,0.000958,0.001044,0.002979,0.000911,0.001046,0.000911,0.000865,0.000982,0.003833,...,0.002117,0.008293,0.012735,0.005601,0.008829,0.006681,0.013458,1.000000,0.007820,0.000998


In [22]:
#similarity = distance['tom']#.sort_values(ascending=False)[1:]
similarity['tom'].sort_values(ascending=False)[1:10]

Name
João Moutinho       0.005831
T. Lemar            0.005827
Raúl García         0.005816
J. Rodríguez        0.005796
G. Bonaventura      0.005791
Koke                0.005768
Anderson Talisca    0.005756
M. Hamšík           0.005753
G. Wijnaldum        0.005750
Name: tom, dtype: float64

In [ ]:
#recom_club.iloc[18207]

In [ ]:
#attributes = recom_club.iloc[:, 0:65]
#attributes.head()

In [ ]:
attributes = pd.concat([attributes], axis=1)
df = attributes
print(attributes.columns)

In [ ]:
scaled = StandardScaler()
X = scaled.fit_transform(attributes)

In [ ]:
recommendations = NearestNeighbors(n_neighbors=6,algorithm='ball_tree')
recommendations.fit(X)

In [ ]:
player_index = recommendations.kneighbors(X)[1]

In [ ]:
player_index

In [ ]:
def get_index(x):
    return club[club['Name']==x].index.tolist()[0]

def recommend_me(player):
    print("5 Players similar to {} are: ".format(player))
    index=  get_index(player)
    for i in player_index[index][1:]:
        print(club.iloc[i]['Name'])

In [ ]:
recommend_me('L. Messi')